# Tidying Up

In the course of presenting certain common problems and short-term solutions in the last chapter, we've introduced some kind of janky code, and this chapter starts with trying to replace them with alternatives that represent better practices.


## Isolation

He speaks first of ensuring that functional tests are *isolated* from one another, but I don't think we're talking (necessarily) about the different functions in the `functional_tests.py` file, but rather the idea of isolating the different instances/calls to the file, so that, as we saw at the end of Chapter 5, we don't have different entries specified by the test piling up in our database file.

For that, the preferred solution is to specify either a separate database from the "production" one be modified by functional tests, or else that any changes logged to it are automatically undone once testing is complete.  Harry says that adding logic to your `setUp` and `tearDown` methods in your functional tests file (again, which extends `unittest.TestCase`), but the cleaner approach is to utilize the `LiveServerTestCase` object that's built into Django since version 1.4.

At this point, he suggests mvoing the functional tests file into a different subdir named the same thing.  You do that via `git`, rather than just the OS alone, to ensure that your repo's tracking of the move is logged and handled appropriately.  You also need to add an (empty) `init` file to the new subdir, to have the Python interpreter treat it like its own module.  Then, Django should parse the files therein and execute any that start with `test` whenever your execute a functional test.

In [2]:
from pathlib import Path

Path('functional_tests').mkdir(exist_ok=True)
with open('functional_tests/__init__.py', 'a') as f:
    pass

In [3]:
# Comment out after executing once:
# !git mv functional_tests.py functional_tests/tests.py

And the new syntax to call for functional tests is 

`python manage.py test functional_tests`

i.e., point to it as a module.

<div class="alert alert-info"><p>There's a box here in the text noting that you <i>could</i> throw your functional test files into that same <code>lists</code> subdir containing the unit tests, but that a lot of times functional tests (which, again, are attempting to simulate what a user might go through when visiting your site) will need to interact with multiple apps in a site, an therefore make sense to organize separately.</p>
<p>I would also note that, at this point, I guess I've been thinking that the "<code>lists</code>" subdir somehow reflected something about how Django tends to organize things, i.e., that "lists" were somehow something universal to web sites in general.  But now that I'm prompted to consider the organization of our project dir anew, I guess I see that it's meant instead to constitute a distinct "app", meaning it simply acknowledges that the point of the main function of our site so far is to store "To-Do" lists.  In general, as I get more experience, it should become more obvious what aspects of the organization of such projects are specific to the way Django expects things to be structured, versus things that we're free to set up however we like, and how to modify our settings and commands accordingly, to have the framework make sense of our projects as we intend.</p></div>

Now, we just modify the renamed functional tests file to utilize the aforementioned `LiveServerTestCase` class.  The other changes made at this point to the file are to remove the reference to the explicit URL and port number in the `retrieve_it_later` method, since this new object has a dedicated `live_server_url` attr that stores such things, and also that we can delete the "`if name`" block at the bottom that renders the file executable as a separate Python script, since we intend to call it in the future via tha `manage.py` command, which will interpret this as a valid set of tests without direct handling from the interpreter:

In [5]:
%%writefile functional_tests/tests.py
from django.test import LiveServerTestCase
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import unittest

class NewVisitorTest(LiveServerTestCase):

    def setUp(self):
        self.browser = webdriver.Firefox()

    def tearDown(self):
        self.browser.quit()

    def check_for_row_in_list_table(self, row_text):
        table = self.browser.find_element_by_id('id_list_table')
        rows = table.find_elements_by_tag_name('tr')
        self.assertIn(row_text, [row.text for row in rows])

    def test_can_start_a_list_and_retrieve_it_later(self):
        # Edith has heard about a cool new online to-do app.  She goes
        # to check out its homepage
        self.browser.get(self.live_server_url)

        # She notices the page title and header mention to-do lists
        self.assertIn('To-Do', self.browser.title)
        header_text = self.browser.find_element_by_tag_name('h1').text
        self.assertIn('To-Do', header_text)

        # She is invited to enter a to-do item straight away
        inputbox = self.browser.find_element_by_id('id_new_item')
        self.assertEqual(
            inputbox.get_attribute('placeholder'),
            'Enter a to-do item'
        )

        # She types "Buy peacock feathers" into a text box 
        # (Edith's hobby is tying fly-fishing lures)
        inputbox.send_keys('Buy peacock feathers')

        # When she hits enter, the page updates, and now the page lists
        # "1: Buy peacock feathers" as an item in a to-do list table
        inputbox.send_keys(Keys.ENTER)
        time.sleep(1)
        self.check_for_row_in_list_table('1: Buy peacock feathers')

        # There is still a text box inviting her to add another item.
        # She enters "Use peacock feathers to make a fly"
        # (Edith is very methodical)
        inputbox = self.browser.find_element_by_id('id_new_item')
        inputbox.send_keys('Use peacock feathers to make a fly')
        inputbox.send_keys(Keys.ENTER)
        time.sleep(1)

        # The page updates again, and now shows both items on her list
        self.check_for_row_in_list_table('1: Buy peacock feathers')
        self.check_for_row_in_list_table('2: Use peacock feathers to make a fly')

        # Edith wonders whether the site will remember her list.  Then she sees
        # ...

Overwriting functional_tests/tests.py


In [6]:
!python manage.py test functional_tests

Creating test database for alias 'default'...
System check identified no issues (0 silenced).
Destroying test database for alias 'default'...


.
----------------------------------------------------------------------
Ran 1 test in 11.970s

OK


At this point in the text there was still a reference to a line like `self.fail('Finish the test!')`, but I don't have that in mine, so it doesn't return any failures at all.  Anyways, add the new subdir to the project with Git, and commit to the remote.


# Handling Waiting is Tricky

Ok, at this point in the text, Harry points out that another kind of wonky workaround applied earlier was the "explicit wait" strategy embodied by the calls to `time.sleep` in the functional tests file.  There is often some kind of load time involved when testing your server (presumably even longer ones when you're actually testing a remote server over some connection like you'd have in a more realistic web app scenario), and the amount of time to wait is tough to anticipate efficiently.  The risk is you underestimate the load time required, in which case you'll prob see `NoSuchElementException`s and `StaleElementException`s; make your tests wait too long, however, and you'll just be reducing the rate at which you can confirm and fix errors, without yielding improved info about what's broken.

He says that Selenium used to try to figure that out for you, employing a strategy called an "implicit wait", but that it got too difficult to implement and unpredictable in its behavior, so they deprecated it as a practice.  Instead, there's this kind of hybrid solution, where Selenium *tries* to figure out how long to wait, but you can set a `MAX_WAIT` constant value that will direct its behavior a little more explicitly.  We then change the name of the `check_for_row_in_list_table` method to `wait_for_row_in_list_table`, make it reference that `MAX_WAIT` constant, and then update the later calls within the huge `test_can_start_a_list...` method to point to the renamed method, and remove the explicit `time.sleep` calls after.

In [8]:
%%writefile functional_tests/tests.py
from django.test import LiveServerTestCase
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import WebDriverException
import time
import unittest

MAX_WAIT = 10

class NewVisitorTest(LiveServerTestCase):

    def setUp(self):
        self.browser = webdriver.Firefox()

    def tearDown(self):
        self.browser.quit()

    def wait_for_row_in_list_table(self, row_text):
        start_time = time.time()
        while True:
            try:
                table = self.browser.find_element_by_id('id_list_table')
                rows = table.find_elements_by_tag_name('tr')
                self.assertIn(row_text, [row.text for row in rows])
                return
            except (AssertionError, WebDriverException) as e:
                if time.time() - start_time > MAX_WAIT:
                    raise e
                time.sleep(0.5)

    def test_can_start_a_list_and_retrieve_it_later(self):
        # Edith has heard about a cool new online to-do app.  She goes
        # to check out its homepage
        self.browser.get(self.live_server_url)

        # She notices the page title and header mention to-do lists
        self.assertIn('To-Do', self.browser.title)
        header_text = self.browser.find_element_by_tag_name('h1').text
        self.assertIn('To-Do', header_text)

        # She is invited to enter a to-do item straight away
        inputbox = self.browser.find_element_by_id('id_new_item')
        self.assertEqual(
            inputbox.get_attribute('placeholder'),
            'Enter a to-do item'
        )

        # She types "Buy peacock feathers" into a text box 
        # (Edith's hobby is tying fly-fishing lures)
        inputbox.send_keys('Buy peacock feathers')

        # When she hits enter, the page updates, and now the page lists
        # "1: Buy peacock feathers" as an item in a to-do list table
        inputbox.send_keys(Keys.ENTER)
        self.wait_for_row_in_list_table('1: Buy peacock feathers')

        # There is still a text box inviting her to add another item.
        # She enters "Use peacock feathers to make a fly"
        # (Edith is very methodical)
        inputbox = self.browser.find_element_by_id('id_new_item')
        inputbox.send_keys('Use peacock feathers to make a fly')
        inputbox.send_keys(Keys.ENTER)

        # The page updates again, and now shows both items on her list
        self.wait_for_row_in_list_table('1: Buy peacock feathers')
        self.wait_for_row_in_list_table('2: Use peacock feathers to make a fly')

        # Edith wonders whether the site will remember her list.  Then she sees
        # ...


Overwriting functional_tests/tests.py


In [9]:
!python manage.py test

Creating test database for alias 'default'...
System check identified no issues (0 silenced).
Destroying test database for alias 'default'...


.......
----------------------------------------------------------------------
Ran 7 tests in 11.506s

OK


At this point, we've had enough successful tests in a row that the author gets a little nervous, and suggests purposely breaking something, to check that it returns the right response.  One such change involves looking for nonsensical text ("`'foo'`") in the functional test's check through the contents of the rows, and another that looks for a nonsensical tag ("`'id_nothing'`") in the browser's content.  I'll skip those here, since it takes up so much space to modify these test files in such a way, just to add a single line.

Oh, ok; that's all for this chapter.  It was a pretty short one.  Commit and move on.